In [5]:
import os
import json

In [6]:
def calculate_intrinsic_value(
    fcf_ps: float,
    growth_rate: float,
    terminal_multiple: float,
    discount_rate: float,
    years: int,
    cash: float,
    debt: float,
    shares: float
) -> float:
    """
    Calculate the intrinsic value per share of a stock using the DCF method,
    assuming `fcf_ps` is the free cash flow per share at year 0.

    Parameters:
    - fcf_ps: The current free cash flow *per share* (e.g., 4.62).
    - growth_rate: The annual growth rate (as a *percentage*, e.g., 8 for 8%).
    - terminal_multiple: The multiple used to value the final year's FCF (e.g., 15).
    - discount_rate: The required annual return (as a *decimal*, e.g., 0.13 for 13%).
    - years: The number of years to project (e.g., 10).
    - cash: The total cash and cash equivalents (not per share).
    - debt: The total debt (not per share).
    - shares: The total number of shares outstanding.

    Returns:
    - The intrinsic value per share (float).
    """
    # Convert growth_rate from percentage (e.g. 8) to decimal (0.08) if needed
    if growth_rate > 0:
        growth_rate = growth_rate / 100.0

    total_value = 0.0

    # Calculate the present value of projected FCF per share for each year
    for year in range(1, years + 1):
        # Projected FCF per share (assuming compound growth)
        fcf_year = fcf_ps * (1 + growth_rate) ** year
        # Discount the FCF back to today
        present_value = fcf_year / ((1 + discount_rate) ** year)
        total_value += present_value

    # Calculate the terminal value per share at the end of the projection period
    fcf_final_year = fcf_ps * (1 + growth_rate) ** years
    terminal_value_per_share = fcf_final_year * terminal_multiple
    terminal_value_pv = terminal_value_per_share / ((1 + discount_rate) ** years)
    total_value += terminal_value_pv

    # Convert total cash and debt into net cash per share
    net_cash_per_share = (cash - debt) / shares

    # Summing discounted FCF per share (enterprise value per share)
    # and net cash per share gives equity value per share
    intrinsic_value_per_share = total_value + net_cash_per_share
    return intrinsic_value_per_share


def intrinsic_values_over_discount_rates(
    fcf_ps: float,
    growth_rate: float,
    terminal_multiple: float,
    years: int,
    cash: float,
    debt: float,
    shares: float,
    start_rate: float = 0.06,
    end_rate: float = 0.18,
    step: float = 0.01
) -> dict:
    """
    Calculate the intrinsic value per share for a range of discount rates.

    Parameters:
    - fcf_ps: Free cash flow per share at year 0.
    - growth_rate: Growth rate (percentage, e.g. 8 for 8%).
    - terminal_multiple: Terminal multiple (e.g., 15).
    - years: Number of years to project (e.g., 10).
    - cash: Total cash (not per share).
    - debt: Total debt (not per share).
    - shares: Total shares outstanding.
    - start_rate: Starting discount rate (decimal, e.g. 0.06 for 6%).
    - end_rate: Ending discount rate (decimal, e.g. 0.18 for 18%).
    - step: Increment in discount rate (e.g. 0.01 -> 1%).

    Returns:
    - A dictionary mapping discount rates (float) to intrinsic value per share (float).
    """
    results = {}
    discount_rate = start_rate

    while discount_rate <= end_rate + 1e-9:  # small tolerance for floating comparison
        iv = calculate_intrinsic_value(
            fcf_ps,
            growth_rate,
            terminal_multiple,
            discount_rate,
            years,
            cash,
            debt,
            shares
        )
        results[discount_rate] = iv
        discount_rate += step

    return results

In [7]:
ticker = 'GOOGL'                    # Ticker
fcf_ps = 60e9 / 13e9                # Initial FCF per share
growth_rate = 8                     # FCF growth
terminal_multiple = 15              # Terminal multiple
years = 10                          # Projection period
cash = 110e9                        # Total cash
debt = 27e9                         # Total debt
shares = 13e9                       # Number of shares outstanding

In [8]:
# Calculate intrinsic values over discount rates from 6% to 18%
intrinsic_values = intrinsic_values_over_discount_rates(
    fcf_ps, 
    growth_rate, 
    terminal_multiple,
    years, 
    cash, 
    debt, 
    shares,
    start_rate=0.06, 
    end_rate=0.18, 
    step=0.01
)

# Print the results
for rate in sorted(intrinsic_values.keys()):
    print(f"{rate*100:.0f}%  ->  ${intrinsic_values[rate]:,.2f}")

# Prepare the results for JSON output (discount rate as percentage)
output_data = {f"{rate*100:.0f}%": intrinsic_values[rate] for rate in intrinsic_values}

# Ensure the 'dcf' directory exists
output_directory = "dcf"
os.makedirs(output_directory, exist_ok=True)

# Create the output file path using the ticker
output_file = os.path.join(output_directory, f"{ticker}.json")

# Write the results to the JSON file
with open(output_file, 'w') as json_file:
    json.dump(output_data, json_file, indent=4)

6%  ->  $141.07
7%  ->  $130.96
8%  ->  $121.77
9%  ->  $113.41
10%  ->  $105.79
11%  ->  $98.84
12%  ->  $92.50
13%  ->  $86.70
14%  ->  $81.40
15%  ->  $76.54
16%  ->  $72.08
17%  ->  $67.99
18%  ->  $64.23
